# Comparing long and short read assemblies

In the context of this course "How to accurately reconstruct mobile genetic elements using bioinformatics", we must consider the limitations of different sequencing platforms, how genome assembly software works, and think about what would be the best approach for exploring MGE. I will also describe methods for assessing genome assemblies as we step through the [simulated dataset](../simulate_reads/simulate_reads.ipynb).



## What is a genome assembler doing?

These days, whole genome sequencing is synomymous with [Shotgun sequencing](https://en.wikipedia.org/wiki/Shotgun_sequencing). In these approaches, the targeted DNA is sheared to an appropriate smaller size. Using these fragements, a sequencing library is prepared that can then be read via sequencing machines. These sequencing machines, for different reasons, have a maximum reading length for each fragment. Today, Illumina individual short reads you will encounter are usually 150 base pairs long, while individual long reads (from ONT or PacBio platforms) can be megabases long. 

To perform de novo genome assembly, these sequencing reads need to have the following properties: 

* They are resolved into nucleotide bases (ATGC & ambiguous base calls)
* They are randomly distributed across the target DNA, and 
* They are represent an oversampling of the target DNA, such that individual reads repeatedly overlap. 

De novo genome assembly software rely on these assumptions to stitch the individual reads into long contiguous sequences (contigs). These software can either directly align the overlaps or produce a graph/network that represent the overlaps. Either way, knowing which reads align to which allows the program to reconstruct the original genome. 

These days, you will encounter "paired end reads", which is where the sequencing machine has read from both ends of the fragment into the centre. The insert is often longer than the machine's reading capability, leaving a part of the target DNA insert unread. This is by design, for reasons we will see later. 

<div>
<img src="read_length.png" width="500"/>
</div>


This a simple (perhaps simplistic) explanation and I encourage you to read further, here is an [Extended review](https://doi.org/10.1093/bib/bbw096) to get started. [PDF](./bbw096.pdf) 

In theory, the software should always reconstruct the original genome perfectly. We can test this if we have perfect simulated reads of a good length. In practice, however, the output is often far from perfect. There can manfiest as a range of artefacts including; false SNPs, false inversions, and false insertions/duplications/deletions. There can be many different reasons why this can occur:

* Some platforms struggle with GC rich and/or AT rich DNA. 
* Some platforms have lower read quality towards the end of reads (3', 5' or both ends) 
* Some platforms have difficulty reading homopolymers (e.g. AAAAA or TTTTTTT) accurately  
* **The read length does not span repeated sequences in the genome**

Illumina platforms have mitigated most assembly issues deriving from the first three problems by providing an absurd amount of sequencing throughput/yield. 

The last problem (limited read length) is something cannot be resolved by sequencing deeper. And it is for this problem we use long read data when we want to recover the original genome sequence verbatim. We recommend a hybrid genome assembly (using short and long reads) 


## Repeats and read length 

Genomes are littered with repetitive elements. They can vary in size and purpose. We recognise them as transposables elements, paralogs, tandem repeats, and the like. The genome assembly software has no such understanding. Instead it sees ambigiuity. When it tries to assemble a repeditive element, it is looking for possible reads that overlap and it finds multiple options. The program only knows for sure that the bases on the same read are connected. This can be extended with the information of the paired end read, because we know those reads - while not overlapping - were physically connected at an approximate distance. This, for Illumina, gives us a maximum window of ~400bp (150 x 2 + the gap in-between), if the repeat is larger than that then there is no definitive way to choose the correct path. When presented with multiple paths, (most) genome assemblers will act conservatively and refusing to make a decision, leave that position as a breakpoint. The diagram below shows the basic resolution from a genome assembler. Usually there is breakpoint either side of the repeat and all the instances of the repeat are represented with a single contig with very high sequencing coverage. We call these "collapsed repeats". We will see how repeats manifest in genome assemblies from our sample data later. 


<div>
<img src="collapsed.jpg" width="500"/>
</div>

[Figure source](https://doi.org/10.1186%2Fgb-2008-9-3-r55)


Keep it mind, this is really down to the length of the repeat sequence against the read length. De novo genome assemblers will (usually) resolve complex repeditive elements like CRISPR, which by definition is "clustered regularly interspaced short palindromic repeats". In this case, the repeats are less than 72bp which is shorter than the read length of 150. Depending on the type of repeat, and how the assembler chooses to handle it, you will see different resulting contigs that may look like the examples below. 

<div>
<img src="excision-rearrangement-consensus.png" width="500"/>
</div>

[Figure source](https://training.galaxyproject.org/training-material/topics/assembly/tutorials/get-started-genome-assembly/slides-plain.html)

I should mention here that resovling repeats may not be a concern for you. If you are willing to completely ignore repeat regions, genome synteny, and focus on gene content (presence/absence) then genome assemblies from short read data may be sufficent. I will next use our simulated read dataset sets to explore the ideas presented above. 


## Difference between assemblies using short read only and long + short 

Let's revisit the assembly metrics of the hybrid (unicycler) assembly and the short read (spades/shovill) assembly. The details of the [read simulation is here](../simulate_reads/simulate_reads.ipynb) and the details of the assembly processes [are here](./assembly_annotation_short_reads.ipynb) and [here](./assembly_annotation_hybrid.ipynb). The number of contigs from the hybrid is 3, which is perfect; we have 1 chromsome and 2 plasmids in our data. The number of contigs in the short read assembly is 240. 

The total assembled length for the hybrid assembly is 5249059 where as the short read assembly is 5204624. Meaning that around 44kb is missing in the short read assembly. In terms of the original genome, the long read assembly has recovered 99.997% of the original chromosome and the short read assembly recovered 98.629%. Considering the other metrics, I would say that the short read assembly is largely workable if you want to explore the genomic content, majority of the genes should have been assembled. It would be difficult to look at genome organisation with the short read assembly, however. And keep in mind that the majority of the breakpoints will be around repeditive elements, which are usually associated with MGE. 

The determining factor of why the Hybrid assembly performed so well is down to the read length from the long reads (ONT). These were generated by a normal distribution where the mean was 15000bp and stdev was 13000. This is the default behaviour for the program I used, badread.  I am unsure the exact length of repeats in E. coli ST131, but 15 kbp seems to exceed all of them given the assembly produced. 


| Assembly                   | Hybrid  | Illumina only | Assembly                    | Hybrid     | Illumina only |
|----------------------------|---------|---------------|-----------------------------|------------|---------------|
| # contigs (>= 0 bp)        | 3       | 240           | LG75                        | 1          | 17            |
| # contigs (>= 1000 bp)     | 3       | 90            | # misassemblies             | 0          | 1             |
| # contigs (>= 5000 bp)     | 2       | 53            | # misassembled contigs      | 0          | 1             |
| # contigs (>= 10000 bp)    | 2       | 42            | Misassembled contigs length | 0          | 320802        |
| # contigs (>= 25000 bp)    | 2       | 30            | # local misassemblies       | 0          | 0             |
| # contigs (>= 50000 bp)    | 2       | 24            | # scaffold gap ext. mis.    | 0          | 0             |
| Total length (>= 0 bp)     | 5249059 | 5204624       | # scaffold gap loc. mis.    | 0          | 0             |
| Total length (>= 1000 bp)  | 5249059 | 5153166       | # unaligned mis. contigs    | 1          | 0             |
| Total length (>= 5000 bp)  | 5245097 | 5064120       | # unaligned contigs         | 1 + 1 part | 8 + 6 part    |
| Total length (>= 10000 bp) | 5245097 | 4988300       | Unaligned length            | 133857     | 129511        |
| Total length (>= 25000 bp) | 5245097 | 4790267       | Genome fraction (%)         | 99.997     | 98.629        |
| Total length (>= 50000 bp) | 5245097 | 4573728       | Duplication ratio           | 1.001      | 1.001         |
| # contigs                  | 3       | 114           | # N's per 100 kbp           | 0          | 0             |
| Largest contig             | 5109618 | 512292        | # mismatches per 100 kbp    | 0.8        | 0.12          |
| Total length               | 5249059 | 5171846       | # indels per 100 kbp        | 0.18       | 0             |
| Reference length           | 5109767 | 5109767       | Largest alignment           | 5109618    | 512292        |
| GC (%)                     | 50.81   | 50.81         | Total aligned length        | 5115202    | 5042335       |
| Reference GC (%)           | 50.77   | 50.77         | NA50                        | 5109618    | 173149        |
| N50                        | 5109618 | 191195        | NGA50                       | 5109618    | 173149        |
| NG50                       | 5109618 | 191671        | NA75                        | 5109618    | 112142        |
| N75                        | 5109618 | 112142        | NGA75                       | 5109618    | 112142        |
| NG75                       | 5109618 | 112142        | LA50                        | 1          | 9             |
| L50                        | 1       | 9             | LGA50                       | 1          | 9             |
| LG50                       | 1       | 8             | LA75                        | 1          | 18            |
| L75                        | 1       | 17            | LGA75                       | 1          | 18            |

I should mention that this result (from simulated reads) is better than what you would expect from real data. Assembly quality can vary greatly and may require multiple sequencing attempts. 


## Comparing our example data visually with Mauve 

We can explore the resulting genome assemblies visually with [Mauve](https://darlinglab.org/mauve/mauve.html). The tool is entirely graphically (point and click) so I can easily feed in the genome data. I have selected the large plasmid sequence (pEC958-HG941719) to act as a reference, and the [plasmid contigs detected by mobsuite](../genotyping/plasmid_reconstruction_mobsuite.ipynb)

<div>
<img src="mauve_input.png" width="300"/>
</div>

The program will perform all the sequence alignments between the assemblies, which took about 30 seconds. This gives us a clear visual of the three assemblies and which regions align to which. Each colour is an alignment block, calculate by Mauve.  Each of the red vertical lines (in the short read assembly at the bottom) are contig breakpoints. These are hard breakpoints for the alignment as well, so you will notice that the number of coloured alignment blocks and contigs are the same (14).

<div>
<img src="mauve_view.png" width="800"/>
</div>

Because we have included the genbank file of pEC958, we can click on the white blocks, which are annotation features from the genbank file and see the details of genes in each of the alignment blocks. In this yellow region there are genes from the *tra* operon.

<div>
<img src="mauve_anno.png" width="300"/>
</div>

On the whole the sequences are effectively the same genetic content. The hybrid assembly looks exactly the same, except it is inverted. This is an excellent result for the short read data. From the short reads we recovered all the genetic content of the plasmid and those contigs were identified sucessfully with MOB-Suite. 

You can use Mauve on your own data. The software is [freely available for all operating systems](https://darlinglab.org/mauve/mauve.html). 

You can also use other tools to do a similar exploration of your data:

* [Artemis genome browser](https://www.sanger.ac.uk/tool/artemis/)
* [Gingr from the Harvest package](https://harvest.readthedocs.io/en/latest/content/gingr.html)
* [IGV](https://software.broadinstitute.org/software/igv/)

In our example, we are lucky to have an exact reference genome to compare. This is unlikely in real datasets, but recall tools like COPLA and MOB-Suite will idenitfy close reference genomes you could use instead. 


# Export quantitative data from Mauve 

It's all very well to look at the Mauve visualisation and conclude that everything is the same, but Mauve also gives discrete data that we can export and use to look at the specific differences between our assemblies

<div>
<img src="mauve_export.png" width="300"/>
</div>

Export SNPs reports there are 5 SNPs across all three genomes.

The gaps file had no output, because there were no gaps.

The permutations file quantifies the order of the alignment blocks. The permutations file looks like this:   

This mirrors what we saw visually; hybrid assembly is the same but inverted (see the "-"). The short read assembly has the same block of sequences but the order cannot be determined due to the contig breakpoints ($). In the permutation file, the alignment blocks are numbered from largest to smallest, hence the number 1, 14, 9, 2 in the reference. 

# Exploring genome graphs with Bandage 

I am trying to avoid getting into the technical aspects - I find that de bruijn graphs have little meaning to most - but looking at the genome assembly graph can be helpful in understanding genome assembly errors. 

We can look at the graph visually with [Bandage](https://rrwick.github.io/Bandage/). The software is freely available for all operating systems. The tool is entirely graphically (point and click) so I can easily feed in the genome data. The graph data is produced as part of the Shovill and Unicycler assemblies as a *.gfa file. 

Here are the graph files from our example data so you can follow along:

* [Short read Shovill assembly graph](../ori/spades_contigs.gfa)
* [Hybrid Unicycler assembly graph](../ori/unicycler_assembly.gfa)

The unicyler graph mirrors what we have seen throughout. Three single sequences, in line with our expectations


<div>
<img src="unicycler_graph.png" width="800"/>
</div>

The shovill graph is more complex, you can see why the assembler hesitated in joining the contigs together. Each of the "beads" are likely collapsed repeats, where the sequences converge into a shared region. As mentioned above this cannot be resolved with short read data alone. If you drag the colored lines, you can start to unravel the graph yourself and inspect which contigs relate to the different features. In my experience, this is a difficult thing to unpack from these data alone. 

<div>
<img src="bandage_spades_graph.png" width="900"/>
</div>

You can explore your data using [PLACNET as well](assembly_plasmid_placnet.ipynb). 

